# Gaussian Process on the concrete UCI dataset, using Inducing Points and all points

# Github

In [1]:
from google.colab import drive # For github
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Project18/GPs
!git config --global user.email "alexander.sabelstrom.1040@student.uu.se"
!git config --global user.name "Sabelz"

Mounted at /content/drive
/content/drive/MyDrive/Project18/GPs


# Imports

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import torch
!pip install gpytorch
import gpytorch

%matplotlib inline
%load_ext autoreload
%autoreload 2
%run ../datasets/concrete.ipynb # Run the toy notebook which is in the datasets folder(toy dataset)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Project18/datasets
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1030 entries, 0 to 1029
Data columns (total 9 columns):
 #   Column                                                 Non-Null Count  Dtype  
---  ------                                                 --------------  -----  
 0   Cement (component 1)(kg in a m^3 mixture)              1030 non-null   float64
 1   Blast Furnace Slag (component 2)(kg in a m^3 mixture)  1030 non-null   float64
 2   Fly Ash (component 3)(kg in a m^3 mixture)             1030 non-null   float64
 3   Water  (component 4)(kg in a m^3 mixture)              1030 non-null   float64
 4   Superplasticizer (component 5)(kg in a m^3 mixture)    1030 non-null   float64
 5   Coarse Aggregate  (component 6)(kg in a m^

# Training/Test data from concrete.ipynb

In [101]:
concrete_data = df_Concrete # df_Concrete is defined in ../datasets/concrete.ipynb
x, y = concrete_data.iloc[:, :-1].to_numpy() , concrete_data.iloc[:, -1].to_numpy()  # The last column is output(concrete compressive strength)
# Normalize
x_mean = x.mean(axis = 0) # mean for each feature
x_std = x.std(axis = 0) # std for each feature
y_mean = y.mean() # mean for output
y_std = y.std() # std for output
x = (x-x_mean) / x_std
y = (y-y_mean) / y_std
# Convert them to tensors
x = torch.from_numpy(x).float()
y = torch.from_numpy(y).float()

# Split into training and test data
len_split = int(0.8 * len(x)) # Where the training points should be split
# The first 80% of points
xTrain = x[:len_split]
yTrain = y[:len_split]
# The last 20% of points
xTest = x[len_split:]
yTest = y[len_split:]

print(xTrain.size(), yTrain.size(), xTest.size(), yTest.size())

torch.Size([824, 8]) torch.Size([824]) torch.Size([206, 8]) torch.Size([206])


# Inducing Points

In [102]:
amount_of_points = 30
inducingPointsX = xTrain[:amount_of_points] # Choose how many points to pick
inducingPointsY = yTrain[:amount_of_points] # Choose how many points to pick
print(len(inducingPointsX))

30


# The GP model

In [103]:
# Class for the GP model(Exact GP)
class GPModel(gpytorch.models.ExactGP):
    def __init__(self, x_Inducing, y_Inducing, likelihood):
        super(GPModel, self).__init__(x_Inducing, y_Inducing, likelihood)
        self.mean_module = gpytorch.means.ConstantMean() # Decide which mean to use
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel()) # Decide which kernel to use
    # GP Posterior predictive distribution
    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# Initialize the first model

In [104]:
# initialize likelihood and model
likelihood = gpytorch.likelihoods.GaussianLikelihood() # Decide likelihood
model = GPModel(inducingPointsX, inducingPointsY, likelihood) # Send in inducing points as the training points
if torch.cuda.is_available():
    model = model.cuda()
    likelihood = likelihood.cuda()

# Training Function

In [105]:
import os
def train(model, xTrain, yTrain): # Train the model on training data: xTrain, yTrain

  smoke_test = ('CI' in os.environ)
  training_iter = 2 if smoke_test else 250


  # Find optimal model hyperparameters
  model.train()
  model.likelihood.train()

  # Use the adam optimizer
  optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters

  # "Loss" for GPs - the marginal log likelihood
  mll = gpytorch.mlls.ExactMarginalLogLikelihood(model.likelihood, model)
  # Train without printing to ensure the training method is as fast as possible
  for i in range(training_iter):
      # Zero gradients from previous iteration
      optimizer.zero_grad()
      # Output from model
      output = model(xTrain)
      # Calc loss and backprop gradients
      loss = -mll(output, yTrain)
      loss.backward()
      optimizer.step()


# Train the Model#

In [106]:
%time train(model, inducingPointsX, inducingPointsY)

CPU times: user 804 ms, sys: 11.5 ms, total: 815 ms
Wall time: 822 ms


# The posterior mean, variance and Covariance Matrix

In [107]:
model.eval() # eval mode is for computing predictions through the model posterior
f_preds = model(xTest) # returns the model posterior distribution p(f* | x*, X, y), for training data X, y
f_mean = f_preds.mean # Predictive mean
f_var = f_preds.variance # Predictive variance
f_covar = f_preds.covariance_matrix # Covariance matrix
print("Mean Dimension: ", f_mean.size())
print()
print("Variance Dimension: ", f_var.size())
print()
print("CovMatrix Dimension ", f_covar.size())

Mean Dimension:  torch.Size([206])

Variance Dimension:  torch.Size([206])

CovMatrix Dimension  torch.Size([206, 206])


# Predictive Distribution

In [108]:
model.eval() # eval mode is for computing predictions through the model posterior.
likelihood.eval()

# Make predictions by feeding model through likelihood
with torch.no_grad(), gpytorch.settings.fast_pred_var(): # https://arxiv.org/abs/1803.06058
    observed_pred = likelihood(model(xTest))# gives us the posterior predictive distribution p(y* | x*, X, y) which is the probability distribution over the predicted output value
    mean = observed_pred.mean.numpy()

# Compare different amount of points

In [111]:
# Trains models one by one for each amount of inducing points, and plots each model with plots [rows,columns](must match the length of listOfPoints)
def severalInducingPoints(listOfPoints):
  maxPoints = len(xTrain) # The max amount of points in training points
  for points in listOfPoints:
    inducingPointsX = xTrain[:points] # Choose how many points to pick
    inducingPointsY = yTrain[:points] # Choose how many points to pick

    # initialize likelihood and model
    likelihood = gpytorch.likelihoods.GaussianLikelihood() # Decide likelihood
    model = GPModel(inducingPointsX, inducingPointsY, likelihood) # Send in inducing points as the training points
    if torch.cuda.is_available():
        model = model.cuda()
        likelihood = likelihood.cuda()
    print()
    print("Inducing Points: ", points)

    %timeit train(model, inducingPointsX, inducingPointsY) # Train the model
    # Plot
    model.eval() # eval mode is for computing predictions through the model posterior.
    likelihood.eval()

severalInducingPoints([5,10,50,100,200,400, 700, 900, 1030]) # With 1030 being all points



Inducing Points:  5
757 ms ± 111 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

Inducing Points:  10
947 ms ± 125 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

Inducing Points:  50
802 ms ± 17.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

Inducing Points:  100
1.23 s ± 138 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

Inducing Points:  200
1.41 s ± 217 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

Inducing Points:  400
3.86 s ± 550 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

Inducing Points:  700
13.6 s ± 640 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

Inducing Points:  900
20.8 s ± 678 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

Inducing Points:  1030
20.9 s ± 762 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
